# Imports

In [1]:
import pandas as pd
from ruqiya import ruqiya
import arabicstopwords.arabicstopwords as stp
from pyarabic.araby import strip_tashkeel
from spacy.lang.ar.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
import plotly.graph_objects as go
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

# EDA

In [42]:
lsa = LsaSummarizer()
df = pd.read_csv('arabic_reviews.csv')

In [3]:
df.head()

,label,text
0,Positive,ممتاز. البوفيه المفتوح رائع جدا سواء في الفطور...
1,Negative,كتاب حكم النبي محمد الذي كتبه الأديب الروائي ا...
2,Positive,معلومات تاريخية عن الثورة الإيرانية وما حدث في...
3,Negative,جلال أمين لم يجد إجابه للسؤال، فأخد يسرد لنا ق...
4,Positive,شقة ممتازة . أعجبني كل شيء وخاصة طاقم العمل..


# Data Cleaning

In [31]:
def preprocess(text):
  text=ruqiya.clean_text(text)
  text=ruqiya.remove_repeating_char(text)
  text=ruqiya.remove_punctuations(text)
  text=ruqiya.normalize_arabic(text)
  text=ruqiya.remove_stop_words(text)
  text=ruqiya.remove_emojis(text)
  return text

In [32]:
df['text'].apply(str)
df['Proccessed'] = df['text'].apply(preprocess)

In [33]:
df.head()

,label,text,Proccessed,Sammurized,done?
0,Positive,ممتاز. البوفيه المفتوح رائع جدا سواء في الفطور...,متاز البوفيه المفتوح راءع سواء الفطور السحور و...,ممتاز. البوفيه المفتوح رائع جدا سواء في الفطور...,Yes
1,Negative,كتاب حكم النبي محمد الذي كتبه الأديب الروائي ا...,كتاب حكم النبي محمد كتبه الاديب الرواءي الروسي...,ولكنه وعد بالكتابة عن النبي في كتاب كامل وللأس...,Yes
2,Positive,معلومات تاريخية عن الثورة الإيرانية وما حدث في...,معلومات تاريخيه الثوره الايرانيه بتفاصيل مله م...,معلومات تاريخية عن الثورة الإيرانية وما حدث في...,Yes
3,Negative,جلال أمين لم يجد إجابه للسؤال، فأخد يسرد لنا ق...,جلال يجد اجابه لسءال فاخد يسرد قصا تسوق استنتج...,فأخد يسرد لنا قصصا تسوق إستنتجات ليست منطقيه. ...,Yes
4,Positive,شقة ممتازة . أعجبني كل شيء وخاصة طاقم العمل..,شقه متازه اعجبني شيء وخاصه طاقم العمل,شقة ممتازة . أعجبني كل شيء وخاصة طاقم العمل..,Yes


# Sammuraize

In [35]:
def sammuraize(text):
    parser = PlaintextParser.from_string(strip_tashkeel(text), Tokenizer("arabic"))

    summary = lsa(parser.document, 2)
    
    return ' '.join([str(sentence) for sentence in summary])

In [36]:
df['Sammurized'] = df['text'].apply(lambda x: sammuraize(x))

/Users/khaledzsa/anaconda3/lib/python3.11/site-packages/sumy/summarizers/lsa.py:76: UserWarning:

Number of words (2) is lower than number of sentences (3). LSA algorithm may not work properly.

/Users/khaledzsa/anaconda3/lib/python3.11/site-packages/sumy/summarizers/lsa.py:76: UserWarning:

Number of words (1) is lower than number of sentences (2). LSA algorithm may not work properly.

/Users/khaledzsa/anaconda3/lib/python3.11/site-packages/sumy/summarizers/lsa.py:76: UserWarning:

Number of words (2) is lower than number of sentences (5). LSA algorithm may not work properly.

/Users/khaledzsa/anaconda3/lib/python3.11/site-packages/sumy/summarizers/lsa.py:76: UserWarning:

Number of words (2) is lower than number of sentences (4). LSA algorithm may not work properly.

/Users/khaledzsa/anaconda3/lib/python3.11/site-packages/sumy/summarizers/lsa.py:76: UserWarning:

Number of words (1) is lower than number of sentences (5). LSA algorithm may not work properly.

/Users/khaledzsa/anaconda

# Validate & Visualize

In [37]:
def aboutsummariz(text:str):
    if len(text)<1:
        ans = 'No'
    else:
        ans = "Yes"
    return ans

In [38]:
df['done?'] = df['Sammurized'].apply(aboutsummariz)

In [39]:
val = dict(df['done?'].value_counts())
labels = list(val.keys())
values = list(val.values())
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])

fig.show()

In [40]:
print("Sammurized: ", len(df['Sammurized'][169]))
print("========================================")
print("Original", len(df['text'][169]))

Sammurized:  282
Original 8063


In [41]:
print("Sammurized: ", df['Sammurized'][169])
print("========================================")
print("Original", df['text'][169])

Sammurized:  لا يمكن إنكار حقيقة جودة اللغة التي كتبت فيها الرواية رغم وجود بعض اللكنة الفرنسية لكن هذا أمر مفهوم لتأثر الكاتب بالثقافة الفرنكوفونية. ولربما تأثر الكاتب خطا بالفترة الأولى من نظرية التحليل النفسي لفرويد حيث كان يعيد كل شيء للدافع الجنسي أو حدث جنسي ما تعرض له المرء أثناء طفولته.
Original الطلياني شكري المبخوت. شكري المبخوت هو كاتب تونسي ورئيس جامعة منوبة التونسية.. له عدة مؤلفات سابقة: سيرة الغائب، وسيرة الآتي وكتاب: جمالية الألفة: النص ومتقبله في التراث النقدي وكتاب: نظرية الأعمال اللغوية .. الطلياني هي أول رواية للمبخوت.. وهي تتحدث عن الشاب عبد الناصر والذي التصقت به كنية الطلياني نظرا لوسامته.. في الواقع هي تتحدث عن سرد في صفحة عن السبب الذي دعا عبد الناصر لضرب الإمام علالة أثناء دفن والده الحاج محمود في حادثة فضائحية لا تنسى في جنازة الشيخ الجليل ومرموق الصيت محمود والد عبد الناصر. وقد تحصلت هذه الرواية على جائزة البوكر لعام .. لا يمكن إنكار حقيقة جودة اللغة التي كتبت فيها الرواية رغم وجود بعض اللكنة الفرنسية لكن هذا أمر مفهوم لتأثر الكاتب بالثقافة الفرنكوفونية.. بد